In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
from tensorflow.keras.layers import Add, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Flatten, Input, LeakyReLU, PReLU, Lambda, MaxPool2D, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras import layers


In [209]:
def res_block_simple(inp, stage, substage):    
    filters=inp.shape.as_list()[-1]//4
    conv=Conv2D(filters, 1, activation='relu', use_bias=False, name='res{0}{1}_branch2a'.format(stage, substage))(inp)   
    conv=Conv2D(filters, 3, padding='same', activation='relu', use_bias=False, name='res{0}{1}_branch2b'.format(stage, substage))(conv)
    conv=Conv2D(filters*4, 1, activation='relu', use_bias=False, name='res{0}{1}_branch2c'.format(stage, substage))(conv)    
    add=Add(name='res{0}{1}_'.format(stage, substage))([inp, conv])
    return ReLU(name='res{0}{1}'.format(stage, substage))(add)

def res_block_ext(inp, stage):    
    filters=inp.shape.as_list()[-1]//2
    strds=(2,2)
    if filters==32:
        filters=filters*2
        strds=(1,1)        
    conv=Conv2D(filters, 1, strides=strds, activation='relu', use_bias=False, name='res{0}a_branch2a'.format(stage))(inp)    
    conv=Conv2D(filters, 3, padding='same', activation='relu', use_bias=False, name='res{0}a_branch2b'.format(stage))(conv)     
    conv=Conv2D(filters*4, 1, use_bias=False, name='res{0}a_branch2c'.format(stage))(conv)    
    inp1=Conv2D(filters*4, 1, strides=strds, use_bias=False, name='res{0}a_branch1'.format(stage))(inp)    
    
    add=Add(name='res{0}a_'.format(stage))([conv,inp1])
    return ReLU(name='res{0}a'.format(stage))(add)

def get_res50(inp_shape=(500,500,3)):
    inp=Input(shape=inp_shape)
    conv=Conv2D(64, 7, padding='same', activation='relu', name='conv1')(inp)    
    out=MaxPool2D(strides=2, name='pool1')(conv)    
    outputs=[conv]
    
    for num, blocks in enumerate([2,3,5,2]):
        stage=num+2
        out=res_block_ext(out, stage)
        for substage in 'bcdef'[:blocks]:
            out=res_block_simple(out, stage, substage)
        outputs.append(out)
    return Model(inputs=inp,outputs=outputs)

In [ ]:
mod=get_res50()
for lr in mod.weights:
    print('{0:20} {1}'.format(lr.name.split('/')[0], lr.shape))

In [ ]:
dot_img_file = 'res50_model.png'
tf.keras.utils.plot_model(mod, to_file=dot_img_file, show_shapes=True)

In [9]:
mod.output

[<tf.Tensor 'conv1_1/Identity:0' shape=(None, 500, 500, 64) dtype=float32>,
 <tf.Tensor 'res2c_1/Identity:0' shape=(None, 250, 250, 256) dtype=float32>,
 <tf.Tensor 'res3d_1/Identity:0' shape=(None, 125, 125, 512) dtype=float32>,
 <tf.Tensor 'res4f_1/Identity:0' shape=(None, 63, 63, 1024) dtype=float32>,
 <tf.Tensor 'res5c_1/Identity:0' shape=(None, 32, 32, 2048) dtype=float32>]

In [208]:
# crop like-caffe layer 
class Crop(layers.Layer):
    """centerred cropping of target layer by dimensions of template layer 
    EXAMPLE:
    cropped_lr=Crop(templ, name='cropped_layer')(target)
    """
    def __init__(self, templ, **kwargs):
        super(Crop, self).__init__(**kwargs)
        self.out_sh = tf.shape(templ)
      

    def call(self, x, trainable=False):
        self.total=tf.zeros(self.out_sh)
        x_shape = tf.shape(x)
        
        offsets = [0, (x_shape[1] - self.out_sh[1]) // 2, (x_shape[2] - self.out_sh[2]) // 2, 0]
        size = [-1, self.out_sh[1], self.out_sh[2], self.out_sh[3]]
        x_crop = tf.slice(x, offsets, size)
        self.total+=x_crop
        return self.total 


# test of cropp layer

In [194]:
inp=Input(shape=(500,500,3))
con=Conv2DTranspose(3,2,(2,2))(inp)
cr=Crop(inp, name='crop_002')(con)
mod=Model(inp,[con,cr])
mod.summary()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

test=np.random.rand(1,500,500,3)
yy=mod.predict(test)

plt.figure(figsize=(20,10))

import matplotlib.pyplot as plt
for num, im in enumerate(yy):
    plt.subplot(1,3,num+1)
    plt.imshow(im[0])
plt.subplot(1,3,3)
plt.imshow(yy[0][0][250:-250,250:-250,:])
dif=np.sum(yy[0][0][250:-250,250:-250,:]-yy[1][0])
plt.title('differense is: {}, crop is valid: {}'.format(dif, dif==0))


# Caffe section

In [ ]:
import caffe
import numpy as np
from PIL import Image
base=r'/home/sgr/COB/models'
PRETRAINED = '{0}/COB_PASCALContext_trainval.caffemodel'.format(base)
MODEL_FILE = '{0}/deploy.prototxt'.format(base)

img = np.asarray(Image.open(r'/home/sgr/Загрузки/COB/demos/2010_005731.png'))

net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)
caffe.set_mode_cpu()

counter=0
for key in net.params:    
    num=net.params[key].__len__()
    out='{0:22}#{1:2}'.format(key, num)
    
    for i in range(num):
        sh=net.params[key][i].data[...].shape
#         out='{0}; {1:3} shape:{2:25}'.format(out, i, sh)
        out='{0};\t{1}: shape:{2:20}'.format(out, i+1, str(sh))
    counter+=1
    print(out)
print('Total params layer: '+str(counter))